## Preparing the Data

In [1]:
# from __future__ import unicode_literals, print_function, division

import glob
import os
import unicodedata
import string

import torch

In [16]:
# Finds {language}.txt files.
def find_files(path):
    return glob.glob(path)

In [34]:
letters = string.ascii_letters + " .,;'"
letters_len = len(letters)

# Unicode -> ascii for names.
def unicode_2_ascii(unicode_string):
    ascii_string = ''
    for char in unicodedata.normalize("NFD", unicode_string):
        # Removes accent marks and non-ascii characters
        if unicodedata.category(char) != "Mn" and char in letters:
            ascii_string += char
         
    return ascii_string

In [35]:
# Gets a list of names from a {language}.txt file.
def read_names(file_name):
    with open(file_name, encoding="utf-8") as f:
        file_data = f.read()
    names = file_data.strip().split('\n')
    
    return [unicode_2_ascii(name) for name in names]

In [36]:
language_names = {}
all_languages = []

for file_name in find_files("data/names/*.txt"):
    language = os.path.splitext(os.path.basename(file_name))[0]
    all_languages.append(language)
    
    names = read_names(file_name)
    language_names[language] = names

languages_len = len(all_languages)

### Turning Names into Tensors

In [37]:
# Letter -> index.
def letter_to_index(letter):
    return letters.find(letter)

In [40]:
# Name -> one-hot array.
def name_to_tensor(name):
    tensor = torch.zeros(len(name), 1, letters_len)
    for i, letter in enumerate(name):
        tensor[i][0][letter_to_index(letter)] = 1
    return tensor

print(name_to_tensor("Jones"))

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0